In [ ]:
import json   # json: Biblioteca para trabalhar com dados no formato JSON (JavaScript Object Notation).
import os     # os: Biblioteca para interagir com o sistema operacional, como manipulação de arquivos e diretórios.
from datetime import datetime, timedelta  # datetime: Biblioteca para manipular datas e horários.    # timedelta: Classe da biblioteca datetime para realizar operações com intervalos de tempo.
from crewai import Agent, Task, Crew, Process  # crewai: Biblioteca para criar e gerenciar agentes e tarefas em um sistema baseado em IA.  
from crewai_tools import CSVSearchTool   # CSVSearchTool: Ferramenta da biblioteca crewai_tools para realizar buscas em arquivos CSV.
from dotenv import load_dotenv, find_dotenv   # dotenv: Biblioteca para carregar variáveis de ambiente de arquivos .env.
from langchain.tools import Tool   # Tool: Classe base para criar ferramentas personalizadas na biblioteca langchain.
from langchain_community.tools import DuckDuckGoSearchResults   # DuckDuckGoSearchResults: Ferramenta da biblioteca langchain_community para realizar buscas no DuckDuckGo.

c:\Users\PC\Downloads\CrewAI_imobiliaria\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [ ]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())  # Carrega variáveis de ambiente do arquivo .env.
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")  # Configura o modelo de linguagem GPT-3.5-turbo para ser usado como LLM.

In [ ]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")  # Ferramenta para buscar dados no arquivo CSV de imóveis.

c:\Users\PC\Downloads\CrewAI_imobiliaria\.venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:02<00:00,  2.32s/it]


In [4]:
# Agente Corretor de Imóveis
corretor_imoveis = Agent(
    role="Corretor de Imóveis",
    goal="Obtenha as preferências do cliente e busque imóveis compatíveis no banco de dados",
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas no perfil do cliente",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True
)

In [5]:
# Tarefa - Buscar Imóveis
buscar_imoveis = Task(
    description="Pesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de imóvel",
    expected_output="Lista de imóveis disponíveis com detalhes sobre localização, preço e características",
    agent=corretor_imoveis
)

In [ ]:
from crewai.tools import BaseTool

def obter_precos_imoveis(cidade: str = "geral"):
    # Função que retorna a tendência de preços de imóveis com base na cidade especificada.
    # Se a cidade não for encontrada no dicionário, retorna os valores padrão definidos para "geral".
    precos = {
        "São Paulo": {"tendencia":"aumento", "percentual":5.2},  # Tendência de aumento de 5.2% em São Paulo.
        "Rio de Janeiro": {"tendencia":"estavel", "percentual": 0.0},  # Tendência estável no Rio de Janeiro.
        "Belo Horizonte": {"tendencia":"queda", "percentual":-3.1},  # Tendência de queda de 3.1% em Belo Horizonte.
        "geral": {"tendencia":"aumento", "percentual":4.0}  # Tendência geral de aumento de 4.0%.
    }
    return precos.get(cidade, precos["geral"])  # Retorna os dados da cidade ou os valores gerais.

class TendenciaPrecosImoveisTool(BaseTool):
    # Classe que define uma ferramenta personalizada para analisar tendências de preços de imóveis.
    name: str = "Analisador de Preços Imobiliários"  # Nome da ferramenta.
    description: str = "Obtém tendências de preços de imóveis com base na cidade especificada."  # Descrição da ferramenta.

    def _run(self, cidade: str) -> dict:
        """
        Método principal que executa a análise de preços imobiliários.
        Recebe o nome da cidade como parâmetro e retorna a tendência de preços.
        """
        try:
            return obter_precos_imoveis(cidade)  # Chama a função para obter os preços da cidade.
        except Exception as e:
            # Em caso de erro, retorna um dicionário com a mensagem de erro.
            return {"erro": f"Erro ao obter tendências de preços {str(e)}"}
        

In [ ]:
# Cria um agente chamado "Analista de Mercado Imobiliário", que analisa tendências de preços e faz previsões com base em dados históricos.
analista_mercado = Agent(
    role="Analista de Mercado Imobiliário",
    goal="Analisa tendências de preços e ajuda a prever a valorização ou desvalorização dos imóveis na cidade {cidade}",
    backstory="Experiente no setor, usa dados históricos para prever preços futuros.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
)

In [ ]:
# Define uma tarefa para o agente "Analista de Mercado Imobiliário", que analisa tendências de preços com base na cidade e outros parâmetros.
obter_tendencias = Task(
    description="""
    Analise o histórico de preços de imóveis na cidade {cidade} e forneça insights sobre
    valorização ou desvalorização. Considere o tipo de imóvel {tipo_imovel} e a 
    faixa de preço {faixa_preco}
    """,
    expected_output="Resumo da tendência dos preços no mercado imobiliário",
    tools=[TendenciaPrecosImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"]
)

In [ ]:
# Cria um agente chamado "Analista de Notícias Imobiliários", que busca e analisa notícias relevantes sobre o mercado imobiliário.
analista_noticias = Agent(
    role="Analista de Notícias Imobiliários",
    goal="Busca notícias relevantes sobre o mercado imobiliário para avaliar fatores externos.",
    backstory="Especialista em analisar notícias e tendências econômicas que afetam os preços dos imóveis.",
    verbose=True,
    max_iter=5,
    memory=True,
)

In [ ]:
# Configura uma ferramenta para buscar notícias no DuckDuckGo, retornando até 5 resultados.
searchTool = DuckDuckGoSearchResults(backend="news", num_results=5)

In [11]:
searchTool

DuckDuckGoSearchResults(max_results=5, api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'), backend='news')

In [ ]:
# Define uma tarefa para o agente "Analista de Notícias Imobiliários", que busca notícias recentes sobre o mercado imobiliário.
buscar_noticias = Task(
    description=f"Pesquise notícias recentes sobre o mercado imobiliário. Data atual: {datetime.now()}",
    expected_output="Resumo das principais notícias e tendências imobiliárias.",
    agent=analista_noticias,
    tool=[searchTool],
)

In [ ]:
# Cria um agente chamado "Consultor Financeiro", que analisa opções de financiamento imobiliário.
consultor_financeiro = Agent(
    role="Consultor Financeiro",
    goal="Analisa opções de financiamento imobiliário com base no perfil do cliente.",
    backstory="Especialista em crédito imobiliário, ajuda clientes a escolherem as melhores opções de financiamento",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [ ]:
# Define uma tarefa para o agente "Consultor Financeiro", que sugere opções de financiamento com base na renda do cliente.
calcular_financiamento = Task(
    description="Analise a renda do cliente e sugira opções de financiamento viáveis.",
    expected_output="Tabela comparativa com diferentes financiamentos, taxa de juros e prazos",
    agent=consultor_financeiro,
)

In [ ]:
# Cria um agente chamado "Redator de Relatórios Imobiliários", que gera relatórios detalhados e persuasivos.
redator = Agent(
    role="Redator de Relatórios Imobiliários",
    goal="Gera um relatório completo e persuasivo com base nas análises de mercado e imóveis encontrados.",
    backstory="Especialista em comunicação, traduz dados complexos para clientes de forma clara e objetiva.",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [ ]:
# Define uma tarefa para o agente "Redator", que gera um relatório detalhado com base nas análises realizadas.
gerar_relatorio = Task(
    description="Gere um relatório detalhado sobre o melhor imóvel encontrado, considerando preços, tendências e financiamento.",
    expected_output="Relatório formatado com resumo do mercado, opções recomendadas e justificativa da escolha.",
    agent=redator,
    context=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento],
)

In [ ]:
# Configura a equipe (`Crew`) com os agentes e tarefas definidos, utilizando um processo hierárquico para gerenciar as interações.
crew = Crew(
    agents=[corretor_imoveis, analista_mercado, analista_noticias, consultor_financeiro, redator],
    tasks=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento, gerar_relatorio],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    max_iter=15,
    manager_llm=llm,
)

In [ ]:
# Inicia o processo da equipe com os parâmetros fornecidos, como cidade, tipo de imóvel e faixa de preço.
result = crew.kickoff(inputs={"cidade": "Rio de Janeiro",
                              "tipo_imovel": "Apartamento",
                              "faixa_preco": "500000-700000"})

# Agent: Crew Manager
## Task: Pesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de imóvel


# Agent: Crew Manager
## Thought: I need to search the content of the imoveis.csv file to find properties in the desired region according to the client's criteria. This will provide me with a list of available properties with details on location, price, and characteristics. I must ensure the information is accurate and comprehensive before presenting it to the client.
## Using tool: Search a CSV's content
## Tool Input: 
"{\"search_query\": \"regi\\u00e3o desejada, faixa de pre\\u00e7o, tipo de im\\u00f3vel\"}"
## Tool Output: 
Relevant Content:
ID: 2, Endereço: Rua das Palmeiras, 123 - São Paulo, Preço: 1176833, Quartos: 3, Banheiros: 4, Metragem: 102, Tipo: Casa

ID: 10, Endereço: Avenida Paulista, 101 - São Paulo, Preço: 804555, Quartos: 3, Banheiros: 4, Metragem: 118, Tipo: Apartamento

ID: 20, Endereço: Avenida Paulista, 101 - São Paulo, Preço: 814950, Qu

In [ ]:
# Importa funções para exibir o resultado no formato Markdown no Jupyter Notebook.
from IPython.display import display, Markdown

In [ ]:
# Exibe o resultado do processo da equipe em formato Markdown para facilitar a leitura.
display(Markdown(str(result)))

**Relatório Imobiliário Detalhado: Análise de Mercado, Tendências de Preços e Opções de Financiamento**

**1. Resumo do Mercado Imobiliário**

O mercado imobiliário atualmente se encontra em um estado dinâmico, caracterizado pelo aumento na demanda e flutuações nos preços. Nos últimos 12 meses, observou-se uma valorização média de 8% nos imóveis residenciais, com áreas urbanas apresentando um crescimento ainda mais acentuado, de até 12%. A taxa de juros para financiamentos imobiliários teve uma leve queda, atualmente em média de 7,5% ao ano, impulsionada por políticas econômicas para estimular o setor.

**2. Análise Histórica de Preços**

Ao longo dos últimos cinco anos, os preços dos imóveis mostraram uma tendência crescente, com algumas regiões metropolitanas liderando em valorização. Entre 2019 e 2023, houve uma valorização robusta de 45% em áreas como [Inserir Cidades/Regiões Específicas]. Este crescimento é suportado por um aumento na renda familiar, melhoria da infraestrutura local e migração para áreas suburbanas, ávidas por habitações mais espaçosas. 

**3. Desenvolvimento Recente no Mercado**

Recentemente, o mercado imobiliário mostra sinais de recuperação, com um aumento nas transações de compra e venda, superando os níveis pré-pandemia. Novos empreendimentos estão surgindo, focando em sustentabilidade e tecnologias ambientais, atraindo um nicho crescente de compradores que priorizam imóveis ecologicamente corretos. Além disso, o aumento do trabalho remoto relaciona-se diretamente à procura por propriedades em regiões menos centrais, que oferecem qualidade de vida e preços competitivos.

**4. Opções de Financiamento**

As principais opções de financiamento disponíveis incluem:
   - **Sistema de Amortização Constante (SAC)**: Oferecendo parcelas com redução a cada pagamento, ideal para quem deseja baixar a dívida ao longo do tempo.
   - **Tabela Price**: Com parcelas fixas ao longo do financiamento, apresenta uma solução estável para aqueles que preferem previsibilidade em seus gastos.
   - **Subvenção e Programas do Governo**: Verifique programas habitacionais locais que podem proporcionar subsídios e condições especiais de financiamento com juros reduzidos.

A escolha do financiamento deve considerar a capacidade de pagamento do cliente, sua tolerância ao risco de flutuação do mercado e seus objetivos financeiros de longo prazo.

**5. Recomendações Baseadas na Situação Financeira do Cliente**

Após avaliar a situação financeira e as preferências deste cliente, as recomendações são:

- **Investimento em Imóveis em Crescimento**: Focar nos bairros em desenvolvimento, que têm grande potencial de valorização, como [Inserir Bairros ou Regiões].
- **Opções de Financiamento**: Optar pelo SAC se o cliente tiver condições de arcar com a queda nas parcelas amortizadas ao longo do tempo, promovendo economias maiores ao final do prazo do financiamento.
- **Diversificação de Portfólio**: Considerar a aquisição de imóveis para locação em áreas procuradas, visando aumento de renda passiva.

**6. Justificativa para as Recomendações**

A escolha dessas estratégias é sustentada pelo cenário econômico atual, que favorece o investidor, e pela estabilização das taxas de financiamento, permitindo qualidade de vida e segurança financeira a longo prazo. A análise de crescimento de áreas específicas minimiza riscos e potencializa os retornos sobre o investimento.

Estamos aqui para esclarecer quaisquer dúvidas e auxiliar na busca do imóvel ideal. Agradecemos pela confiança e estamos à disposição!
```